In [30]:
import os
import sys
module_path = os.path.abspath(os.path.join('../src/nigep/layers'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from keras.utils import get_file
import pandas as pd
from keras.layers import Dense
from keras.models import Sequential
from src.nigep import Nigep
from src.nigep.layers.salt_and_pepper_noise import SaltAndPepperNoise
import zipfile
from keras.layers import Flatten, Dense, Conv2D, MaxPooling2D, Dropout
from keras.models import Sequential, Model
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.optimizers.legacy import Adam, SGD, RMSprop

In [31]:
dataset_url = "https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip"
data_dir = get_file(
    origin=dataset_url, fname="./dog_cat.zip", cache_dir='.'
)
with zipfile.ZipFile('./datasets/dog_cat.zip', 'r') as zip_ref:
    zip_ref.extractall('./dataset')

In [32]:
from keras.preprocessing.image import ImageDataGenerator

model = Sequential()

model.add(SaltAndPepperNoise(ratio=.2, input_shape=(150, 150, 3)))

model.add(Conv2D(16, (5, 5), padding="same", activation="relu",  input_shape=(150, 150, 3)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(32, (5, 5), padding="same", activation="relu"))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (5, 5), padding="same", activation="relu"))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

model.add(Dense(512, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

model.build()
model.summary()
model.compile(optimizer=RMSprop(0.001), loss='binary_crossentropy', metrics=['accuracy'])

generator = ImageDataGenerator(rescale=1./255)

train_gen = generator.flow_from_directory('./dataset/cats_and_dogs_filtered/train', target_size=(150,150), batch_size=20)
val_gen = generator.flow_from_directory('./dataset/cats_and_dogs_filtered/validation', target_size=(150,150), batch_size= 20)


history = model.fit(
    train_gen,
    batch_size=20,
    validation_data=val_gen,
    epochs=30
)


Model: "sequential_22"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_51 (Conv2D)          (None, 150, 150, 16)      1216      
                                                                 
 max_pooling2d_50 (MaxPooli  (None, 75, 75, 16)        0         
 ng2D)                                                           
                                                                 
 conv2d_52 (Conv2D)          (None, 75, 75, 32)        12832     
                                                                 
 max_pooling2d_51 (MaxPooli  (None, 37, 37, 32)        0         
 ng2D)                                                           
                                                                 
 conv2d_53 (Conv2D)          (None, 37, 37, 64)        51264     
                                                                 
 max_pooling2d_52 (MaxPooli  (None, 18, 18, 64)      

KeyboardInterrupt: 